<a href="https://colab.research.google.com/github/yoraoui/PFE2022/blob/main/banino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
!pip install tensorflow_lattice
import tensorflow_lattice as tfl
import keras
from tensorflow.keras.layers import AbstractRNNCell


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.0/242.0 kB 3.3 MB/s eta 0:00:00


In [3]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Layer, LSTMCell
from tensorflow.keras.layers import LSTM
task_env_size = 2.2
task_neurons_seed = 8341
task_targets_type = 'softmax'
task_lstm_init_type = 'softmax'
task_n_pc = [256]
task_pc_scale = [0.01]
task_lstm_init_type = 'softmax'
task_n_hdc = [12]
task_hdc_concentration = [20.]
task_dataset_info =  'square_room'
task_root = ''
training_minibatch_size = 10
model_nh_lstm = 128
model_nh_bottleneck = 256
model_dropout_rates = [0.5]
model_weight_decay = 1e-5
model_bottleneck_has_bias = False
model_init_weight_disp = 0.0
nh_lstm = 128

task_neurons_seed = 8341
rs = np.random.RandomState(task_neurons_seed)
concentration = 20
stdev = 0.35
means_hd = rs.uniform(-np.pi, np.pi, (task_n_hdc[0]))
kappa = np.ones_like(means_hd) * concentration
pos_min = -5
pos_max = 5
means_pc = rs.uniform(pos_min, pos_max, size=(task_n_pc[0], 2))
variances = np.ones_like(means_pc) * stdev**2


In [25]:
from keras.src.layers import RNN
from keras.src import backend

class MinimalRNNCell(AbstractRNNCell):

  def __init__(self, units, **kwargs):
    self.units = units
    super(MinimalRNNCell, self).__init__(**kwargs)

  @property
  def state_size(self):
    return self.units

  def build(self, input_shape):
    self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                  initializer='uniform',
                                  name='kernel')
    self.recurrent_kernel = self.add_weight(
        shape=(self.units, self.units),
        initializer='uniform',
        name='recurrent_kernel')
    self.built = True

  def call(self, inputs, states):
    prev_output = states[0]
    h = backend.dot(inputs, self.kernel)
    output = h + backend.dot(prev_output, self.recurrent_kernel)
    return output, output

In [19]:
def get_place_cell_ensembles(
    env_size, neurons_seed, targets_type, lstm_init_type, n_pc, pc_scale):
  """Create the ensembles for the Place cells."""
  place_cell_ensembles = [
        PlaceCellEnsemble(
          n,
          stdev=s,
          pos_min=-env_size / 2.0,
          pos_max=env_size / 2.0,
          seed=neurons_seed,
          soft_targets=targets_type,
          soft_init=lstm_init_type)
      for n, s in zip(n_pc, pc_scale)
  ]
  return place_cell_ensembles


def get_head_direction_ensembles(
    neurons_seed, targets_type, lstm_init_type, n_hdc, hdc_concentration):
  """Create the ensembles for the Head direction cells."""
  head_direction_ensembles = [
      HeadDirectionCellEnsemble(
          n,
          concentration=con,
          seed=neurons_seed,
          soft_targets=targets_type,
          soft_init=lstm_init_type)
      for n, con in zip(n_hdc, hdc_concentration)
  ]
  return head_direction_ensembles


place_cell_ensembles = get_place_cell_ensembles(
  env_size=task_env_size,
  neurons_seed=task_neurons_seed,
  targets_type=task_targets_type,
  lstm_init_type=task_lstm_init_type,
  n_pc=task_n_pc,
  pc_scale=task_pc_scale)

head_direction_ensembles = get_head_direction_ensembles(
    neurons_seed=task_neurons_seed,
    targets_type=task_targets_type,
    lstm_init_type=task_lstm_init_type,
    n_hdc=task_n_hdc,
    hdc_concentration=task_hdc_concentration)


In [20]:
input_tensors = []
init_pos = tf.random.uniform(shape=(10, 2))
init_hd = tf.random.uniform(shape=(10, 1))
ego_vel = tf.random.uniform(shape=(10, 3))
target_pos = tf.random.uniform(shape=(1, 2))
target_hd = tf.random.uniform(shape=(1, 1))
vel_noise = tfp.distributions.Normal(0.0, 1.0).\
sample(sample_shape=ego_vel.get_shape()) * 0.2
input_tensors = [ego_vel + vel_noise] + input_tensors
initial_conds = []
x_pc = init_pos[:, tf.newaxis, :]
diff = x_pc[:, :, tf.newaxis, :] - means_pc[np.newaxis, np.newaxis, ...]
y = -0.5 * tf.reduce_sum((diff**2)/ variances, axis=-1)
lp = y - tf.reduce_logsumexp(y)
print(lp.shape)
lp_reshaped = tf.reshape(lp, (256,10))
init_pc = tf.nn.softmax(lp_reshaped)
init_pc = tf.squeeze(init_pc)

x_hd = init_hd[:, tf.newaxis, :]
diff = kappa * tf.cos(x_hd - means_hd[np.newaxis, np.newaxis, :])
lp = diff - tf.reduce_logsumexp(diff)
lp_reshaped = tf.reshape(lp, (12,10))
init_hd = tf.nn.softmax(lp_reshaped)
init_hd = tf.squeeze(init_hd)


x_targ_pc = target_pos
diff = x_targ_pc - means_pc
y = -0.5 * tf.reduce_sum((diff**2)/ variances, axis=-1)
lp = y - tf.reduce_logsumexp(y)
init_targ_pos = tf.nn.softmax(lp_reshaped)


diff = kappa*tf.cos(target_hd-means_hd[np.newaxis, np.newaxis, :])
lp = y - tf.reduce_logsumexp(diff)
init_targ_hd = tf.nn.softmax(lp)


(10, 1, 256)


In [21]:
# Copyright 2018 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Ensembles of place and head direction cells.

These classes provide the targets for the training of grid-cell networks.

"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from typing import Any

import numpy as np
import tensorflow as tf


def one_hot_max(x, axis=-1):
  """Compute one-hot vectors setting to one the index with the maximum value."""
  return tf.one_hot(tf.argmax(x, axis=axis),
                    depth=x.get_shape()[-1],
                    dtype=x.dtype)


def softmax(x, axis=-1):
  """Compute softmax values for each sets of scores in x."""
  return tf.nn.softmax(x, dim=axis)


def softmax_sample(x):
  """Sample the categorical distribution from logits and sample it."""
  dist = tf.contrib.distributions.OneHotCategorical(logits=x, dtype=tf.float32)
  return dist.sample()


class CellEnsemble(object):
  """Abstract parent class for place and head direction cell ensembles."""

  def __init__(self, n_cells, soft_targets, soft_init):
    self.n_cells = n_cells
    if soft_targets not in ["softmax", "voronoi", "sample", "normalized"]:
      raise ValueError
    else:
      self.soft_targets = soft_targets
    # Provide initialization of LSTM in the same way as targets if not specified
    # i.e one-hot if targets are Voronoi
    if soft_init is None:
      self.soft_init = soft_targets
    else:
      if soft_init not in [
          "softmax", "voronoi", "sample", "normalized", "zeros"
      ]:
        raise ValueError
      else:
        self.soft_init = soft_init

  def get_targets(self, x):
    """Type of target."""

    if self.soft_targets == "normalized":
      targets = tf.exp(self.unnor_logpdf(x))
    elif self.soft_targets == "softmax":
      lp = self.log_posterior(x)
      targets = softmax(lp)
    elif self.soft_targets == "sample":
      lp = self.log_posterior(x)
      targets = softmax_sample(lp)
    elif self.soft_targets == "voronoi":
      lp = self.log_posterior(x)
      targets = one_hot_max(lp)
    return targets

  def get_init(self, x):
    """Type of initialisation."""

    if self.soft_init == "normalized":
      init = tf.exp(self.unnor_logpdf(x))
    elif self.soft_init == "softmax":
      lp = self.log_posterior(x)
      init = softmax(lp)
    elif self.soft_init == "sample":
      lp = self.log_posterior(x)
      init = softmax_sample(lp)
    elif self.soft_init == "voronoi":
      lp = self.log_posterior(x)
      init = one_hot_max(lp)
    elif self.soft_init == "zeros":
      init = tf.zeros_like(self.unnor_logpdf(x))
    return init

  def loss(self, predictions, targets):
    """Loss."""

    if self.soft_targets == "normalized":
      smoothing = 1e-2
      loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=(1. - smoothing) * targets + smoothing * 0.5,logits=predictions,name="ensemble_loss")
      loss = tf.reduce_mean(loss, axis=-1)
    else:
      loss = tf.nn.softmax_cross_entropy_with_logits(
          labels=targets,
          logits=predictions,
          name="ensemble_loss")
    return loss

  def log_posterior(self, x):
    logp = self.unnor_logpdf(x)
    log_posteriors = logp - tf.reduce_logsumexp(logp, axis=2, keep_dims=True)
    return log_posteriors


class PlaceCellEnsemble(CellEnsemble):
  """Calculates the dist over place cells given an absolute position."""

  def __init__(self, n_cells, stdev=0.35, pos_min=-5, pos_max=5, seed=None,
               soft_targets=None, soft_init=None):
    super(PlaceCellEnsemble, self).__init__(n_cells, soft_targets, soft_init)
    # Create a random MoG with fixed cov over the position (Nx2)
    rs = np.random.RandomState(seed)
    self.means = rs.uniform(pos_min, pos_max, size=(self.n_cells, 2))
    self.variances = np.ones_like(self.means) * stdev**2

  def unnor_logpdf(self, trajs):
    # Output the probability of each component at each point (BxTxN)
    diff = trajs[:, :, tf.newaxis, :] - self.means[np.newaxis, np.newaxis, ...]
    unnor_logp = -0.5 * tf.reduce_sum((diff**2)/ self.variances, axis=-1)
    return unnor_logp
  def __call__(self, *args: Any, **kwds: Any) -> Any:
    return super().__call__(*args, **kwds)


class HeadDirectionCellEnsemble(CellEnsemble):
  """Calculates the dist over HD cells given an absolute angle."""

  def __init__(self, n_cells, concentration=20, seed=None,
               soft_targets=None, soft_init=None):
    super(HeadDirectionCellEnsemble, self).__init__(n_cells,
                                                    soft_targets,
                                                    soft_init)
    # Create a random Von Mises with fixed cov over the position
    rs = np.random.RandomState(seed)
    self.means = rs.uniform(-np.pi, np.pi, (n_cells))
    self.kappa = np.ones_like(self.means) * concentration

  def unnor_logpdf(self, x):
    return self.kappa * tf.cos(x - self.means[np.newaxis, np.newaxis, :])
  def __call__(self, *args: Any, **kwds: Any) -> Any:
    return super().__call__(*args, **kwds)



In [ ]:
# RNN layer can take a list of cells, which will then stack them together.
# By default, keras RNN will only return the last timestep output and will not
# return states. If you need whole time sequence output as well as the states,
# you can set `return_sequences` and `return_state` to True.
rnn_layer = tf.keras.layers.RNN([tf.keras.layers.LSTMCell(128),
                                 tf.keras.layers.LSTMCell(256)],
                                return_sequences=True,
                                return_state=True)
outputs, output_states = rnn_layer(inputs, states)

In [9]:
task_env_size = 2.2
task_neurons_seed = 8341
task_targets_type = 'softmax'
task_lstm_init_type = 'softmax'
task_n_pc = [256]
task_pc_scale = [0.01]
task_lstm_init_type = 'softmax'
task_n_hdc = [12]
task_hdc_concentration = [20.]
task_dataset_info =  'square_room'
task_root = ''
training_minibatch_size = 10
model_nh_lstm = 128
model_nh_bottleneck = 256
model_dropout_rates = [0.5]
model_weight_decay = 1e-5
model_bottleneck_has_bias = False
model_init_weight_disp = 0.0

rnn_core = GridCellsRNNCell(
  target_ensembles=target_ensembles,
  nh_lstm=model_nh_lstm,
  nh_bottleneck=model_nh_bottleneck,
  dropoutrates_bottleneck=np.array(model_dropout_rates),
  bottleneck_weight_decay=model_weight_decay,
  bottleneck_has_bias=model_bottleneck_has_bias,
  init_weight_disp=model_init_weight_disp)


NameError: ignored

In [11]:
place_cell_ensembles = get_place_cell_ensembles(
  env_size=task_env_size,
  neurons_seed=task_neurons_seed,
  targets_type=task_targets_type,
  lstm_init_type=task_lstm_init_type,
  n_pc=task_n_pc,
  pc_scale=task_pc_scale)

head_direction_ensembles = get_head_direction_ensembles(
    neurons_seed=task_neurons_seed,
    targets_type=task_targets_type,
    lstm_init_type=task_lstm_init_type,
    n_hdc=task_n_hdc,
    hdc_concentration=task_hdc_concentration)


target_ensembles = place_cell_ensembles + head_direction_ensembles
rnn_core = GridCellsRNNCell(
  target_ensembles=target_ensembles,
  nh_lstm=model_nh_lstm,
  nh_bottleneck=model_nh_bottleneck,
  dropoutrates_bottleneck=np.array(model_dropout_rates),
  bottleneck_weight_decay=model_weight_decay,
  bottleneck_has_bias=model_bottleneck_has_bias,
  init_weight_disp=model_init_weight_disp)


NameError: ignored

In [27]:

inputs_pc = tf.keras.Input(shape=(init_pc.shape[0]*init_pc.shape[1],))
d = tf.keras.layers.Dense(init_pc.shape[0]*init_pc.shape[1])
x_pc = d(inputs_pc)

inputs_hd = tf.keras.Input(shape=(init_hd.shape[0]*init_hd.shape[1],))
d = tf.keras.layers.Dense(init_hd.shape[0]*init_hd.shape[1])
x_hd = d(inputs_hd)
initialization_pc = tf.keras.layers.Dense(128)(x_pc)
initialization_hd = tf.keras.layers.Dense(128)(x_hd)

layer_pc = tfl.layers.Linear(num_input_dims=init_pc.shape[0]*init_pc.shape[1],units=128)
layer_hd = tfl.layers.Linear(num_input_dims=init_hd.shape[0]*init_hd.shape[1],units=128)
ego_vel = tf.random.uniform(shape=(10, 10, 3))
number_of_units=128
n_time_steps, n_features=10, 10

# 'outputs' is a tensor of shape [batch_size, max_time, cell_state_size]
initialization = tf.concat([initialization_pc, initialization_hd], 0)

# Assuming you have a SimpleRNNCell defined as rnn_core
rnn_cell = MinimalRNNCell(units=128)

# 'ego_vel' is a tensor of shape [batch_size, n_time_steps, n_features]
ego_vel = tf.random.uniform(shape=(35, 10, 3))  # Adjust batch_size as needed

# 'initialization_pc' is a tensor of shape [batch_size, 128]

# Use tf.keras.layers.RNN wrapper
rnn_layer = tf.keras.layers.RNN(rnn_cell, return_sequences=True, return_state=True)

# 'state' is a tensor of shape [batch_size, cell_state_size]
outputs, state = rnn_layer(ego_vel, initial_state=[initialization])



In [ ]:
initialization = tf.concat([initialization_pc, initialization_hd], 0)

new class of RNN Cell


In [ ]:
class GridCellsRNNCell(keras.layers.Layer):
  def __init__(self,
               target_ensembles,
               nh_lstm,
               nh_bottleneck,
               nh_embed=None,
               dropoutrates_bottleneck=None,
               bottleneck_weight_decay=0.0,
               bottleneck_has_bias=False,
               init_weight_disp=0.0,
               name="grid_cells_core",  **kwargs):
    self._lstm = LSTM(nh_lstm)
    self.state_size = 128
    super(GridCellsRNNCell, self).__init__(**kwargs)

    self._target_ensembles = target_ensembles
    self._nh_embed = nh_embed
    self._nh_lstm = nh_lstm
    self._nh_bottleneck = nh_bottleneck #256
    self._dropoutrates_bottleneck = dropoutrates_bottleneck
    self._bottleneck_weight_decay = bottleneck_weight_decay
    self._bottleneck_has_bias = bottleneck_has_bias
    self._init_weight_disp = init_weight_disp
    #self.training = False

  def __call__(self, inputs, prev_state):
    conc_inputs = tf.concat(inputs, axis=1, name="conc_inputs")
    lstm_inputs = conc_inputs
    lstm_output, next_state = self._lstm(lstm_inputs, prev_state)
    bottleneck = Dense(self._nh_bottleneck,
                            use_bias=self._bottleneck_has_bias,
                            regularizers={
                                "w": tf.contrib.layers.l2_regularizer(
                                    self._bottleneck_weight_decay)},
                            name="bottleneck")(lstm_output)


    if self.training and self._dropoutrates_bottleneck is not None:
      tf.logging.info("Adding dropout layers")
      n_scales = len(self._dropoutrates_bottleneck)  #model_dropout_rates=0.5
      scale_pops = tf.split(bottleneck, n_scales, axis=1)
      dropped_pops = [tf.nn.dropout(pop, rate, name="dropout")
                      for rate, pop in zip(self._dropoutrates_bottleneck,
                                           scale_pops)]
      bottleneck = tf.concat(dropped_pops, axis=1)
    # Outputs
    ens_outputs = [snt.Linear(
        ens.n_cells,
        regularizers={
            "w": tf.contrib.layers.l2_regularizer(
                self._bottleneck_weight_decay)},
        initializers={
            "w": displaced_linear_initializer(self._nh_bottleneck,
                                              self._init_weight_disp,
                                              dtype=tf.float32)},
        name="pc_logits")(bottleneck)
                   for ens in self._target_ensembles]
    return (ens_outputs, bottleneck, lstm_output), tuple(list(next_state))
  def get_initial_state(self, batch_size, dtype):
    # Your custom get_initial_state implementation here
    # Return a tensor with the appropriate shape and dtype
    initial_state = tf.zeros([batch_size, self.state_size], dtype=dtype)
    return initial_state
  @property
  def output_size(self):
    """Returns a description of the output size, without batch dimension."""
    return tuple([ens.n_cells for ens in self._target_ensembles] +
                 [self._nh_bottleneck, self._nh_lstm])





In [ ]:
from keras.src.layers import RNN, LSTM
from keras.src import backend

# First, let's define a RNN Cell, as a layer subclass.
class GridRNNCell(keras.layers.Layer):

    def __init__(self, units, **kwargs):
        self.units = units
        self.state_size =  (self.units, self.units)
        self._lstm = tf.keras.layers.LSTM(32)
        super(GridRNNCell, self).__init__(**kwargs)

    def build(self, input_shape):
        pass

    def call(self, inputs, prev_state):
        #conc_inputs = tf.concat(inputs, axis=1, name="conc_inputs")
        lstm = LSTM(32, return_state=True)(inputs)
        y = lstm(inputs, initial_state=prev_state)
        return None
        return lstm_output


# Let's use this cell in a RNN layer:
x = tf.keras.Input(shape = (10, 5))  # 10 timesteps, input dim 5
# Define previous state tensors
previous_hidden_state = tf.keras.Input((32,))
previous_cell_state = tf.keras.Input((32,))
previous_state = [previous_hidden_state, previous_cell_state]
cell = GridRNNCell(32)
layer = RNN(cell)
y = layer(x, previous_state)
